Simpletransformers kütüphanesini indiriyoruz.

In [ ]:
pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 K

Gerekli Kütüphaneleri dahil ediyoruz.

In [ ]:
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel

Datamızı yüklüyoruz.

In [ ]:
data_path = 'data.csv' 
a = pd.read_csv(data_path, delimiter='|')
a.dropna(subset=['text'], inplace=True)

Datadaki etiketlere bakıyoruz.

In [ ]:
a["target"].unique()

array(['INSULT', 'RACIST', 'SEXIST', 'PROFANITY', 'OTHER'], dtype=object)

Sınıfları sayısal değere dönüştürüyoruz.

In [ ]:
a['labels'] = pd.factorize(a.target)[0]

Datayı Train ve Test olmak üzere iki gruba ayırıyoruz

In [ ]:

from sklearn.model_selection import train_test_split

train, test = train_test_split(a, test_size=0.1, random_state=42)
     

In [ ]:
train=train[["text","labels"]]
test=test[["text","labels"]]
train["text"]=train["text"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

Bert ile sınıflandırma 

In [ ]:
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=5, use_cuda=False,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3, "train_batch_size": 64 , "fp16":False,"output_dir":"bert_model"})

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Modeli eğitme

In [ ]:
model.train_model(train)

: 

Modelin Doğruluğunu test etme

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)
predictions = model_outputs.argmax(axis=1)
actuals = test.labels.values

In [ ]:
actuals[:10]

In [ ]:
predictions[:10]

Modeli Kaydetme

In [ ]:
model_name = "turkish_bert_model"
model_path = f"/path/to/saved_model_directory/{model_name}"
model.save_model(model_path)